# Let's bring in all the scraped data

In [60]:
import csv
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
# read in all the job postings

corpusDF = pd.DataFrame()

for f in os.listdir("collected_data/"):
    if (f.endswith(".csv")):
        
        print("Importing... ", f)
        
        filepath = os.path.join("collected_data", f)
        
        df = pd.read_csv(filepath, encoding = "ISO-8859-1")
        
        df["AllDescription"] = df["PositionTitle"] + " | " +  \
                                df["URI"] + " | " + df["DepartmentName"]  + " | " + \
                                df["OrganizationName"] + " | " + f  + " | " + \
                                df["JobSummary"] + " | " + \
                                df["QualificationSummary"]
        
        corpusDF = corpusDF.append(df, ignore_index = True)        
        
        print("Success")
        print("================================================================")

Importing...  aerospace_N_113_20171224133539.csv
Success
Importing...  analysis_N_500_20171223141842.csv
Success
Importing...  analyst_N_239_20171223115544.csv
Success
Importing...  attorney_N_180_20171223141257.csv
Success
Importing...  aviation_N_355_20171224133743.csv
Success
Importing...  businessintelligence_N_20_20171223115447.csv
Success
Importing...  combat_N_400_20171224133814.csv
Success
Importing...  computational_N_17_20171223115812.csv
Success
Importing...  covert_N_4_20171223141634.csv
Success
Importing...  cyber_N_76_20171223141650.csv
Success
Importing...  dataanalyst_N_96_20171223115356.csv
Success
Importing...  dataanalytics_N_19_20171223115427.csv
Success
Importing...  database_N_500_20171223120145.csv
Success
Importing...  dataengineer_N_101_20171223115828.csv
Success
Importing...  datamining_N_9_20171223120312.csv
Success
Importing...  datascience_N_346_20171223115408.csv
Success
Importing...  datascientist_N_28_20171223115414.csv
Success
Importing...  datavisualiz

Success
Importing...  sensor_N_23_20171224133831.csv
Success
Importing...  sixsigma_N_6_20171223141604.csv
Success
Importing...  software_N_500_20171223115557.csv
Success
Importing...  soldier_N_162_20171224133432.csv
Success
Importing...  spatial_N_14_20171223115245.csv
Success
Importing...  SPSS_N_3_20171223120435.csv
Success
Importing...  sql_N_21_20171223120239.csv
Success
Importing...  statistical_N_277_20171223120834.csv
Success
Importing...  statistician_N_8_20171223115854.csv
Success
Importing...  statistics_N_329_20171223115916.csv
Success
Importing...  strategic_N_339_20171224133904.csv
Success
Importing...  surgeon_N_58_20171224133621.csv
Success
Importing...  systems_N_500_20171223120219.csv
Success
Importing...  tableau_N_2_20171223120948.csv
Success
Importing...  tactical_N_142_20171224133851.csv
Success
Importing...  technical_N_500_20171223115709.csv
Success
Importing...  technician_N_500_20171223141315.csv
Success
Importing...  technology_N_500_20171223120810.csv
Succe

In [62]:
corpusDF.head(30)

,ApplicationCloseDate,DepartmentName,JobGrade,JobSummary,MaxPay,MinPay,OfferingType,OrganizationName,PayType,PositionEndDate,PositionID,PositionLocation,PositionStartDate,PositionTitle,QualificationSummary,URI,AllDescription
0,2018-01-08,National Aeronautics and Space Administration,GS,The Applied Engineering and Technology Directo...,86460.0,66510.0,Permanent,Goddard Space Flight Center,Per Year,2018-01-08,GS18D0058,"Greenbelt, Maryland",2017-12-20,"Aerospace Engineer, AST, Aerospace Vehicle Des...",In addition to the Basic Education Requirement...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"Aerospace Engineer, AST, Aerospace Vehicle Des..."
1,2017-12-28,Department of the Navy,GS,The selectee for this position will serve as a...,72901.0,45970.0,Permanent,"U.S. Atlantic Fleet, Commander in Chief",Per Year,2017-12-28,DE-10078062-18-HB,"Point Mugu, California",2017-12-18,AEROSPACE ENGINEER,"In order to qualify for this position, your re...",https://www.usajobs.gov:443/GetJob/ViewDetails...,AEROSPACE ENGINEER | https://www.usajobs.gov:4...
2,2017-12-30,Department of the Air Force,GS,The mission of the United States Air Force is ...,86460.0,45970.0,Permanent,"Air Force Elements, U.S. Strategic Command",Per Year,2017-12-30,9L-10088723-064321,"Dahlgren, Virginia",2017-12-01,Aerospace Engineer,GS-0861-07 Aerospace Engineer:\r\nA Bachelor&#...,https://www.usajobs.gov:443/GetJob/ViewDetails...,Aerospace Engineer | https://www.usajobs.gov:4...
3,2017-12-29,National Aeronautics and Space Administration,GS,"The Flight Projects Directorate, Restore-L Pro...",161900.0,131767.0,Permanent,Goddard Space Flight Center,Per Year,2017-12-29,GS18C0041,"Greenbelt, Maryland",2017-12-13,"Aerospace Engineer, AST, Aerospace Flight Systems",In addition to the Basic Education Requirement...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"Aerospace Engineer, AST, Aerospace Flight Syst..."
4,2018-01-02,National Aeronautics and Space Administration,GS,As a technical lead in the Revolutionary Aviat...,155073.0,119285.0,Permanent,Langley Research Center,Per Year,2018-01-02,LA18D0008,"Hampton, Virginia",2017-12-19,"Research Aerospace Engineer, AST Aerospace Fli...",Applicant must have one year of specialized ex...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"Research Aerospace Engineer, AST Aerospace Fli..."
5,2017-12-30,Department of the Air Force,GS,The mission of the United States Air Force is ...,86460.0,45970.0,Permanent,"Air Force Elements, U.S. Strategic Command",Per Year,2017-12-30,9L-10088771-079552-MDA,"Dahlgren, Virginia",2017-12-01,AEROSPACE ENGINEER,GS-0861-07 Aerospace Engineer:\r\nA Bachelor&#...,https://www.usajobs.gov:443/GetJob/ViewDetails...,AEROSPACE ENGINEER | https://www.usajobs.gov:4...
6,2018-01-08,National Aeronautics and Space Administration,GS,"The Flight Projects Directorate, Global Ecosys...",161900.0,131767.0,Permanent,Goddard Space Flight Center,Per Year,2018-01-08,GS18C0076,"Greenbelt, Maryland",2017-12-20,"Aerospace Engineer, AST, Aerospace Flight Systems",In addition to the Basic Education Requirement...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"Aerospace Engineer, AST, Aerospace Flight Syst..."
7,2018-01-08,National Aeronautics and Space Administration,GS,Note: This position is also being announced un...,125706.0,81319.0,Permanent,Armstrong Flight Research Center,Per Year,2018-01-08,AF18D0007,"Edwards AFB, California",2017-12-18,Aerospace Engineer,Applicant must have one year of specialized ex...,https://www.usajobs.gov:443/GetJob/ViewDetails...,Aerospace Engineer | https://www.usajobs.gov:4...
8,2018-01-08,National Aeronautics and Space Administration,GS,The Suborbital and Special Orbital Projects Di...,155073.0,101409.0,Permanent,Goddard Space Flight Center,Per Year,2018-01-08,GS18C0070,"Wallops Island, Virginia",2017-12-13,"Aerospace Engineer, AST, Aerospace Flight Systems",In addition to the Basic Education Requirement...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"Aerospace Engineer, AST, A

## Top 10 departments with the highest average paying jobs

In [63]:
set(corpusDF["PayType"])

{'Bi-weekly',
 'Fee Basis',
 'Per Day',
 'Per Hour',
 'Per Year',
 'Student Stipend Paid',
 'Without Compensation'}

In [64]:
print("Total Jobs: ", len(corpusDF))

Total Jobs:  18426


For ease of analysis, I'm only going to look at "Per Year" jobs with the assumption that these salaried positions typically pay higher than the hourly positions. 

In [65]:
salaried = corpusDF[corpusDF["PayType"] == "Per Year"]
print("Total Salaried Jobs: ", len(salaried))

Total Salaried Jobs:  14447


There are also some positions with multiple location placements. I'm going to filter those out. They have a single pipe delimitor in the "PositionLocation" column. 

In [70]:
salaried = salaried[salaried["PositionLocation"].str.contains(' | ') == True]
salaried.head(10)

,ApplicationCloseDate,DepartmentName,JobGrade,JobSummary,MaxPay,MinPay,OfferingType,OrganizationName,PayType,PositionEndDate,PositionID,PositionLocation,PositionStartDate,PositionTitle,QualificationSummary,URI,AllDescription,MedPay
1,2017-12-28,Department of the Navy,GS,The selectee for this position will serve as a...,72901.0,45970.0,Permanent,"U.S. Atlantic Fleet, Commander in Chief",Per Year,2017-12-28,DE-10078062-18-HB,"Point Mugu, California",2017-12-18,AEROSPACE ENGINEER,"In order to qualify for this position, your re...",https://www.usajobs.gov:443/GetJob/ViewDetails...,AEROSPACE ENGINEER | https://www.usajobs.gov:4...,59435.5
7,2018-01-08,National Aeronautics and Space Administration,GS,Note: This position is also being announced un...,125706.0,81319.0,Permanent,Armstrong Flight Research Center,Per Year,2018-01-08,AF18D0007,"Edwards AFB, California",2017-12-18,Aerospace Engineer,Applicant must have one year of specialized ex...,https://www.usajobs.gov:443/GetJob/ViewDetails...,Aerospace Engineer | https://www.usajobs.gov:4...,103512.5
11,2018-03-31,Department of the Air Force,GS,The mission of the United States Air Force is ...,155073.0,32844.0,Multiple Appointment Types,Air Force Personnel Center,Per Year,2018-03-31,AFPC-ACQEHA-10085705-0861,"Eielson AFB, Alaska|Elmendorf AFB, Alaska|Fort...",2017-12-01,AEROSPACE ENGINEERING SERIES,BASIC REQUIREMENT:\r\nDegree: Professional Eng...,https://www.usajobs.gov:443/GetJob/ViewDetails...,AEROSPACE ENGINEERING SERIES | https://www.usa...,93958.5
14,2018-11-09,Department of the Air Force,NH,The mission of the United States Air Force is ...,134776.0,18526.0,Multiple Appointment Types,Air Force Materiel Command,Per Year,2018-11-09,AFMC-10077875-DHA-NH0861-JL,"Davis Monthan AFB, Arizona|Davis Monthan AFB, ...",2017-11-10,Aerospace Engineer - Direct Hire Authority,EXPERIENCE: Applicants will be rated in accord...,https://www.usajobs.gov:443/GetJob/ViewDetails...,Aerospace Engineer - Direct Hire Authority | h...,76651.0
21,2018-11-07,Department of the Navy,GS,The selectee for this position will serve as a...,133967.0,32844.0,Multiple Appointment Types,U.S. Navy - Agency Wide,Per Year,2018-11-07,DE-10080002-18-LS,"Phoenix, Arizona|Tucson, Arizona|Yuma, Arizona...",2017-11-08,INTERDISCIPLINARY (ENGINEER),"In order to qualify for this position, your re...",https://www.usajobs.gov:443/GetJob/ViewDetails...,INTERDISCIPLINARY (ENGINEER) | https://www.usa...,83405.5
30,2018-01-22,Department of Defense,GS,Join Team DCMA -- Challenge yourself! The Defe...,112665.0,86663.0,Permanent,Defense Contract Management Agency,Per Year,2018-01-22,SWH817P8XXXXXX1007EA,"Sunnyvale, California",2017-12-12,Interdisciplinary Engineer/Operations Research...,"In order to qualify, you must meet the educati...",https://www.usajobs.gov:443/GetJob/ViewDetails...,Interdisciplinary Engineer/Operations Research...,99664.0
31,2017-12-31,Department of the Air Force,DR,The Air Force's only organization wholly dedic...,134776.0,38371.0,Multiple Appointment Types,Air Force Materiel Command,Per Year,2017-12-31,9X17EHA-2007791-000000-XB,"Edwards AFB, California|Eglin AFB, Florida|Egl...",2017-09-29,"Professional Engineers, DR-08XX-1/2/3/4",SPECIALIZED EXPERIENCE: At least one (1) year ...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"Professional Engineers, DR-08XX-1/2/3/4 | http...",86573.5
32,2018-01-31,Department of the Air Force,GS,"""These positions are being filled under the Ac...",69207.0,56229.0,Permanent,Air Force Personnel Center,Per Year,2018-01-31,BE18EHA-ENGINEER2010329-JDC,"Gunter AFB, Alabama|Maxwell AFB, Alabama|Edwar...",2017-09-25,"PALACE ACQUIRE INTERN-ENGINEER, GS-8XX-9-FY18",Experience refers to paid and unpaid experienc...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"PALACE ACQUIRE INTERN-ENGINEER, GS-8XX-9-FY18 ...",62718.0
33,2018-01-31,Department of the Air Force,GS,"""These positions are being filled under the Ac...",56015.0,45512.0,Permanent,Air Force Personnel Center,Per Year,2018-01-3

In [67]:
salaried["MedPay"] = salaried[["MinPay", "MaxPay"]].mean(axis = 1)
salaried.head(3)

,ApplicationCloseDate,DepartmentName,JobGrade,JobSummary,MaxPay,MinPay,OfferingType,OrganizationName,PayType,PositionEndDate,PositionID,PositionLocation,PositionStartDate,PositionTitle,QualificationSummary,URI,AllDescription,MedPay
0,2018-01-08,National Aeronautics and Space Administration,GS,The Applied Engineering and Technology Directo...,86460.0,66510.0,Permanent,Goddard Space Flight Center,Per Year,2018-01-08,GS18D0058,"Greenbelt, Maryland",2017-12-20,"Aerospace Engineer, AST, Aerospace Vehicle Des...",In addition to the Basic Education Requirement...,https://www.usajobs.gov:443/GetJob/ViewDetails...,"Aerospace Engineer, AST, Aerospace Vehicle Des...",76485.0
1,2017-12-28,Department of the Navy,GS,The selectee for this position will serve as a...,72901.0,45970.0,Permanent,"U.S. Atlantic Fleet, Commander in Chief",Per Year,2017-12-28,DE-10078062-18-HB,"Point Mugu, California",2017-12-18,AEROSPACE ENGINEER,"In order to qualify for this position, your re...",https://www.usajobs.gov:443/GetJob/ViewDetails...,AEROSPACE ENGINEER | https://www.usajobs.gov:4...,59435.5
2,2017-12-30,Department of the Air Force,GS,The mission of the United States Air Force is ...,86460.0,45970.0,Permanent,"Air Force Elements, U.S. Strategic Command",Per Year,2017-12-30,9L-10088723-064321,"Dahlgren, Virginia",2017-12-01,Aerospace Engineer,GS-0861-07 Aerospace Engineer:\r\nA Bachelor&#...,https://www.usajobs.gov:443/GetJob/ViewDetails...,Aerospace Engineer | https://www.usajobs.gov:4...,66215.0


In [68]:
# For each DepartmentName, average out the MinPay, MedPay, and MaxPay

AvgMinPay = salaried.groupby("DepartmentName")["MinPay"].mean().to_frame().sort_values("MinPay", ascending = False)

AvgMinPay["MinPay"] = AvgMinPay["MinPay"].round()

AvgMinPay

,MinPay
DepartmentName,
Department of Labor,114092.0
Department of Energy,105423.0
Department of Housing And Urban Development,105131.0
National Foundation on the Arts and the Humanities,100137.0
National Aeronautics and Space Administration,99635.0
Department of the Treasury,98023.0
Non-Federal Civilian Customers,94796.0
Department of Transportation,80524.0
Judicial Branch,79782.0
